In [31]:
import pickle
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Params

In [10]:
data_dir = 'data'

# Launch web page in Chrome

In [82]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.cametan.com')



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/ryotanomura/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


# Load info

## Prefecture page

In [8]:
prefecture_dict = {}
area_ul_list = driver.find_elements_by_css_selector("div.content-area > ul")

for ul in area_ul_list:
    area_name = ul.text
    area_a_list = ul.find_elements_by_css_selector('a')
    
    for a in area_a_list:
        prefecture_dict[a.text] = a.get_attribute('href')

# with open(f'{data_dir}/prefecture_dict.pickle', mode='wb') as f:
#     pickle.dump(prefecture_dict, f)

## Municipalities page

In [18]:
mncp_dict = {}

for p_name, p_url in prefecture_dict.items():
    
    if p_name not in mncp_dict:
        mncp_dict[p_name] = {}
    
    mncp_list_url = p_url.replace("area/", "area/mncp/")
    driver.get(mncp_list_url)
    
    # mncp without links will be ignored
    mncp_elems = driver.find_elements_by_css_selector('ul.gyo-area > li > a')
    for elem in mncp_elems:
        mncp_dict[p_name][elem.text] = elem.get_attribute('href')
        
# with open(f'{data_dir}/mncp_dict.pickle', mode='wb') as f:
#     pickle.dump(mncp_dict, f)

## Detail page

In [37]:
def search_lat_lon(text):
    return re.search(r'[0-9]+.[0-9]+,[0-9]+.[0-9]+', text).group(0)

In [55]:
loc_dict = {}

for p_name, p_elem in mncp_dict.items():
    if p_name not in loc_dict:
        loc_dict[p_name] = {}
    
    for m_name, m_url in p_elem.items():
        if m_name not in loc_dict[p_name]:
            loc_dict[p_name][m_name] = {}
            
        count = 0
        driver.get(m_url)
        
        gmap_url = driver.find_element_by_css_selector('div.city-map > iframe').get_attribute('src')
        coordinates = search_lat_lon(gmap_url)
        
        # get url
        loc_elems = driver.find_elements_by_css_selector('div.rank-wrap')
        for loc in loc_elems:
            loc_dict[p_name][m_name][count] = {
                'url': loc.find_element_by_css_selector('div > div > a').get_attribute('href'),
                'text': loc.text
            }
            count += 1
        
        loc_b_elems = driver.find_elements_by_css_selector('div.rankb-wrap')
        for loc in loc_b_elems:
            loc_dict[p_name][m_name][count] = {
                'url': loc.find_element_by_css_selector('div > a').get_attribute('href'),
                'text': loc.text
            }
            count += 1

# with open(f'{data_dir}/loc_dict.pickle', mode='wb') as f:
#     pickle.dump(loc_dict, f)

## External page

In [65]:
def is_internal(url):
    return url.startswith('https://www.cametan.com')

In [83]:
ext_dict = {}

for p_name, p_elem in loc_dict.items():
    if p_name not in ext_dict:
        ext_dict[p_name] = {}
        
    for m_name, m_elem in p_elem.items():
        if m_name not in ext_dict[p_name]:
            ext_dict[p_name][m_name] = {}
        
        for index, loc_elem in m_elem.items():
            url = loc_elem['url']
            if is_internal(url):
                driver.get(url)
                
                ext_dict[p_name][m_name][index] = {
                    'url': driver.find_element_by_css_selector('div.guide > a').get_attribute('href'),
                    'text': driver.find_element_by_css_selector('div.head-right').text
                }
                
# with open(f'{data_dir}/ext_dict.pickle', mode='wb') as f:
#     pickle.dump(ext_dict, f)

## Aggregate data

In [96]:
agg_dict = {}

for p_name, p_elem in loc_dict.items():
    if p_name not in agg_dict:
        agg_dict[p_name] = {}
        
    for m_name, m_elem in p_elem.items():
        if m_name not in agg_dict[p_name]:
            agg_dict[p_name][m_name] = {}
            
        for index, loc_elem in m_elem.items():
            url = loc_elem['url']
            if is_internal(url):
                agg_dict[p_name][m_name][index] = {
                    'url': ext_dict[p_name][m_name][index]['url'],
                    'text': loc_elem['text'],
                    'text-detail': ext_dict[p_name][m_name][index]['text']
                }
            else:
                agg_dict[p_name][m_name][index] = loc_elem
                
# with open(f'{data_dir}/agg_dict.pickle', mode='wb') as f:
#     pickle.dump(agg_dict, f)